In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/av-football-hack-rating-prediction/data/sample_submission.csv
/kaggle/input/av-football-hack-rating-prediction/data/Analyticsvidhya-football-hack-2022-06-12.pdf
/kaggle/input/av-football-hack-rating-prediction/data/train.csv
/kaggle/input/av-football-hack-rating-prediction/data/test.csv


In [2]:
! pip install scikit-learn --upgrade --force-reinstall

     |████████████████████████████████| 24.8 MB 26.2 MB/s            
     |████████████████████████████████| 306 kB 50.1 MB/s            
     |████████████████████████████████| 38.1 MB 61.2 MB/s            
     |████████████████████████████████| 15.7 MB 50.7 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.0.0
    Uninstalling threadpoolctl-3.0.0:
      Successfully uninstalled threadpoolctl-3.0.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2


In [3]:
! pip install --no-deps autoxgb

In [4]:
from autoxgb import AutoXGB

In [5]:
train_filename = "../input/av-football-hack-rating-prediction/data/train.csv"
test_filename = "../input/av-football-hack-rating-prediction/data/test.csv"

In [6]:
output = 'autoxgb'
targets = ['rating_num']
task = 'regression'

In [7]:
use_gpu = True
num_folds = 5
seed = 42
num_trials = 100
time_limit = 5*60
fast = True

In [8]:


model = AutoXGB(
    train_filename=train_filename,
    output=output,
    test_filename=test_filename,
    task=task,
    targets=targets,
    use_gpu=use_gpu,
    num_folds=num_folds,
    idx='row_id',
    seed=seed,
    num_trials=num_trials,
    time_limit=time_limit,
    fast=fast)

2022-06-12 20:38:05,214 INFO Output directory: autoxgb


In [9]:
model.train()

2022-06-12 20:38:08,635 INFO Reading training data
2022-06-12 20:38:28,459 INFO Mem. usage decreased to 34.43 Mb (72.4% reduction)
2022-06-12 20:38:28,460 INFO Problem type: single_column_regression
/opt/conda/lib/python3.7/site-packages/autoxgb/autoxgb.py:145: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[self.idx] = np.arange(len(df))
2022-06-12 20:38:34,970 INFO Mem. usage decreased to 14.76 Mb (72.4% reduction)
2022-06-12 20:38:34,972 INFO Creating folds
/opt/conda/lib/python3.7/site-packages/autoxgb/autoxgb.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragment

In [10]:
test_preds = pd.read_csv("./autoxgb/test_predictions.csv")

In [11]:
test_preds.head()

,id,rating_num
0,0,6.746451
1,1,4.747578
2,2,5.740692
3,3,6.193942
4,4,6.113321


In [12]:
test_preds.pop('id')

0          0
1          1
2          2
3          3
4          4
        ... 
8769    8769
8770    8770
8771    8771
8772    8772
8773    8773
Name: id, Length: 8774, dtype: int64

In [13]:
test_preds.describe()

,rating_num
count,8774.000000
mean,6.461500
std,0.962828
min,0.517512
25%,5.813611
50%,6.440504
75%,7.116950
max,9.610333


In [14]:
sample = pd.read_csv("../input/av-football-hack-rating-prediction/data/sample_submission.csv")
sample.head()

,row_id,rating_num
0,14785,7
1,15125,7
2,13095,7
3,6079,7
4,16584,7


In [15]:
sample['rating_num'] = test_preds['rating_num']
#test_preds.to_csv("my_submission_file.csv", index=False)

In [16]:
sample['rating_num'].value_counts()

5.788728    4
4.888780    4
6.334430    2
7.278301    2
6.046708    2
           ..
7.249332    1
5.840134    1
5.733892    1
6.892087    1
7.219878    1
Name: rating_num, Length: 8760, dtype: int64

In [19]:
sample.to_csv("model_1.csv", index=False)